# 要約 
このノートブックは、Kaggleのコンペティション「LMSYS - Chatbot Arena」において、ユーザーの好みに基づくチャットボット応答の優劣を予測するための推論プロセスの最適化を目指したものです。具体的には、LLM（大規模言語モデル）であるLlamaを使用し、応答確率を計算する際の推論速度を38%向上させています。

### 取り組んでいる問題
ノートブックは、主に次の問題に焦点を当てています:
1. **推論時間の短縮**: 元のスクリプトの65分から、40分に推論時間を短縮することに成功。
2. **動的パディングの実装**: 入力データの長さに基づいてバッチ内でパディングを動的に適用し、冗長なパディングを減少させています。
3. **入力シーケンスの最大長の設定**: モデルが長い入力に対処できるように、`max_length`を1024から2048に拡張。

### 使用している手法・ライブラリ
- **フレームワーク**: PyTorchが使用され、特にGPUリソースを活用したモデルの推論が行われています。
- **Transformersライブラリ**: Hugging FaceのTransformersライブラリを利用してLLMを読み込み、トークナイズ、モデルの推論を実施。
- **テストタイム拡張（TTA）**: モデルの出力を改善するために、異なる応答の順序を入れ替える手法も試されていますが、効果は限定的でした。
- **メモリ効率の良い注意機構**: メモリ使用量を削減するために、Memory-Efficient Attentionを有効化しています。

### 実装の流れ
1. **データの準備**: テストデータをトークナイズし、整形します。
2. **モデルの設定と読み込み**: 複数のGPUにモデルを読み込み、LoRA（Low-Rank Adaptation）による微調整を設定します。
3. **推論プロセス**: データフレームをバッチ処理し、各モデルによる出力を計算し、結果を結合して最終的な確率を算出します。
4. **結果の保存**: モデルによる勝者の確率を提出用CSVファイルに保存します。

このノートブックは、チャットボットの応答を評価するための効率的な推論手法の実践的なケーススタディを示しており、その中で最善のプラクティスが紹介されています。

---


# 用語概説 
以下に、Jupyter Notebookの内容から初心者がつまずきそうな専門用語について簡単に解説します。特に、あまり一般的でない用語やこのノートブック特有のドメイン知識に焦点を当てて説明します。

1. **動的パディング**
   - すべての入力シーケンスを同じ長さにするために、短いシーケンスにゼロなどのトークンを追加するプロセスです。固定長ではなく、各ミニバッチ内の最長シーケンスの長さに合わせてパディングすることで、無駄な計算を減らし、効率を向上させることが可能です。

2. **テストタイム拡張（TTA: Test Time Augmentation）**
   - テスト段階でデータを増やす手法で、モデルの予測を改善するために、同一サンプルを異なる方法（ここでは応答の順序の入れ替え）で処理することにより、より多様な予測を得るアプローチです。

3. **ソフトマックス**
   - 多クラス分類問題で使う活性化関数で、出力を確率として解釈できるように変換します。一つの出力を1にし、他の出力を0から1の間の数値に変換します。このノートでは、モデルから得られたロジット（raw scores）に対して適用されています。

4. **メモリ効率の良い注意機構**
   - 大規模モデルを扱う際にメモリ消費を抑えるための特別な設計の注意機構です。通常の注意機構は計算が重くなるため、特に大規模トークン数に対して効率的な方法が求められます。この機構では、メモリの使用を最小限に抑え、計算時間を短縮します。

5. **LoRA（Low-Rank Adaptation）**
   - モデルのパラメータをカスタマイズするための手法で、全体の重みを変更するのではなく、少数の追加のパラメータを用いてモデルをファインチューニングする方法です。元のモデルのパラメータが固定され、新しいタスクに特化した軽量の適応を可能にします。

6. **バイアス**
   - 機械学習モデルで、学習や推論が特定の方向に偏ることを指します。ここではバイアスの設定を指しており、特定の学習条件や目的に基づいて調整されます。

7. **トークナイズ**
   - テキストデータをモデルが理解できる形式（トークン）に変換するプロセスです。トークンは単語やサブワードなど、意味のある単位であり、それにより自然言語処理（NLP）モデルがテキストを処理できるようになります。

8. **最長の長さでパディング（padding="longest"）**
   - ミニバッチ内の最も長いシーケンスに応じて他のシーケンスをパディングする方法です。バッチ全体の効率を向上させるために選ばれることが多いです。

これらの説明は、初心者が踏まえておくべき用語の理解を助け、特にこのノートブックにおける重要な概念が良く分かる助けになるでしょう。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## 🦙🦙🦙 What this notebook is
This notebook is made upon [Inference - llama-3 8b](https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b) by @kishanvavdara. If you haven't checked the linked notebook I highly recommend you to check and upvote.
I made a few improvements upon @kishanvavdara's work:

### 38% faster inference
Inference time using the first 10k samples in the training set takes 40 mins using this script (without TTA) while the original script takes 65 mins, which is 38% faster without any degradation in accuracy. I mainly added two things:

#### 1. Dynamic padding
Instead of padding all the inputs to a fixed length in advance, padding is applied on-the-fly up to the longest sequence in each mini-batch.

#### 2. Sort the test data by input length
To take full advantage of dynamic padding, the test data is sorted by input length. This way, inputs in each mini-batch have more or less same length to reduce the redundant padding.

### Longer input sequence
Although 99% of the training data falls within 1024, the rest 1% are not. Besides, test set may have more long sequences, so I suppose it's safer to make `max_length` as long as possible.
Changing `max_length` from 1024 to 1280 improved LB from 0.989 to 0.983.

## Things I have tried but didn't work

### Test Time Augmentation (TTA)
I tried a simple TTA which swaps the order of response_a and response_b. Note that this will increase the inference time by 2x as model is called twice per sample.
We can average the two softmax probabilities or average the two logits and then compute softmax probability. Alghouth both approaches didn't improve LB, averaging softmax performed better.
TTA will increase the inference time 2x as model is called twice per sample. Submission finished within 9 hours with `max_length=1280` and TTA enabled thanks to the efficient inference.

### Truncate each input
The original implementation truncates the concatenated sequence i.e. prompt + response_a + response_b. Naively applying truncation may end up producing prompt only input as some (though rare) prompt is longer than 1280 tokens, then the model has no way but randomly guessing the winner.
I tried to truncate each input to a fixed length first and then concatenate the three. But it didn't improve LB.

## 🆕 Update in version 4
The efficient inference gives us enough time to increase the input sequence length, so I changed `max_length` to 2048 while mini-batch size is reduced to 4 from 8.
In addition, I enabled [Memory-Efficient Attention](https://github.com/facebookresearch/xformers) to reduce memory usage.
This improved LB from 0.983 to 0.979 and submission still takes less then 4 hours without TTA.
We can go even longer by reducing mini-batch size to 1 but I haven't tested yet.

# Import libs

</div>
<div class="column-right">

# 日本語訳

## 🦙🦙🦙 このノートブックの内容
このノートブックは、@kishanvavdaraによる[Inference - llama-3 8b](https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b)を基に作成されています。リンク先のノートブックをまだ確認していない場合は、ぜひチェックして評価をお願いします。
私は、@kishanvavdaraの作業をもとにいくつかの改善を加えました：

### 38%速い推論
このスクリプトを使って、トレーニングセットの最初の10,000サンプルに対する推論時間は40分で、一方で元のスクリプトは65分かかるため、精度に影響を与えずに38%速くなっています。主に次の2つを追加しました：

#### 1. 動的パディング
すべての入力を事前に固定長にパディングする代わりに、各ミニバッチ内の最長シーケンスまで動的にパディングが適用されます。

#### 2. テストデータを入力長でソート
動的パディングの利点を最大限に活かすため、テストデータは入力長でソートされます。これにより、各ミニバッチ内の入力がほぼ同じ長さになり、冗長なパディングを減らすことができます。

### より長い入力シーケンス
訓練データの99%は1024以内に収まっていますが、残りの1%は収まっていません。さらに、テストセットにはより長いシーケンスが含まれる可能性があるため、`max_length`は可能な限り長く設定する方が安全だと思います。
`max_length`を1024から1280に変更することでLBが0.989から0.983に改善されました。

## 試してみたが効果がなかったこと

### テストタイム拡張（TTA）
私は、response_aとresponse_bの順序を入れ替えるシンプルなTTAを試しました。これにより、サンプルごとにモデルが2回呼び出されるため推論時間が2倍になります。
二つのソフトマックス確率を平均化するか、二つのロジットを平均化してからソフトマックス確率を計算できます。両方のアプローチではLBが改善されませんでしたが、ソフトマックスの平均化がより良い結果を示しました。
TTAはサンプルごとにモデルを2回呼び出すため、推論時間が2倍になります。`max_length=1280`とTTAを有効にして提出が完了しましたが、効率的な推論により9時間以内に収まりました。

### 各入力を切り詰める
元の実装では、プロンプト + response_a + response_bという連結シーケンスが切り詰められます。直截的な切り詰めを適用すると、一部の（稀ではありますが）プロンプトが1280トークンを超えるため、モデルは勝者をランダムに推測するしかなくなります。
私は、最初に各入力を固定長に切り詰めてから、三つを連結しようとしましたが、LBは改善されませんでした。

## 🆕 バージョン4の更新
効果的な推論のおかげで入力シーケンスの長さを増やす時間が十分にあるため、`max_length`を2048に変更しました。また、ミニバッチサイズは8から4に減少させました。
さらに、メモリ使用量を減らすために[Memory-Efficient Attention](https://github.com/facebookresearch/xformers)を有効にしました。
これによりLBが0.983から0.979に改善され、提出はTTAなしで4時間未満で済みました。
ミニバッチサイズを1に減らすことでさらに長くできますが、まだテストしていません。

# ライブラリのインポート


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
</details>

In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, LlamaForSequenceClassification, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
```

</div>
<div class="column-right">

# 日本語訳

```python
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, LlamaForSequenceClassification, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
```

</div>
</details>

In [ ]:
import time
from dataclasses import dataclass
from concurrent.futures import ThreadPoolExecutor

import torch
import sklearn
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, LlamaForSequenceClassification, BitsAndBytesConfig
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

According to the pytorch [documentation](https://pytorch.org/docs/master/generated/torch.nn.functional.scaled_dot_product_attention.html?highlight=scaled_dot_product#torch.nn.functional.scaled_dot_product_attention), `scaled_dot_product_attention` automatically select the most optimal implementation from:
1. Flash Attention
2. Memory Efficient Attention
3. A PyTorch (naive) implementation

By default, all of those are enabled but we can also manually enable/disable certain backends.

</div>
<div class="column-right">

# 日本語訳

PyTorchの[ドキュメント](https://pytorch.org/docs/master/generated/torch.nn.functional.scaled_dot_product_attention.html?highlight=scaled_dot_product#torch.nn.functional.scaled_dot_product_attention)によると、`scaled_dot_product_attention`は、自動的に最も最適な実装を選択します：
1. Flash Attention
2. メモリ効率の良い注意機構
3. PyTorchの（ナイーブな）実装

デフォルトでは、これらすべてが有効ですが、特定のバックエンドを手動で有効化/無効化することもできます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
assert torch.cuda.device_count() == 2, "Sorry - multi-GPU required!"
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)  # Doesn't have any effect as Flash Attention does not support T4/P100
```

</div>
<div class="column-right">

# 日本語訳

```python
assert torch.cuda.device_count() == 2, "申し訳ありませんが、マルチGPUが必要です！"
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)  # Flash AttentionはT4/P100をサポートしていないため、効果はありません。
```

</div>
</details>

In [ ]:
assert torch.cuda.device_count() == 2, "申し訳ありませんが、マルチGPUが必要です！"
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)  # Flash AttentionはT4/P100をサポートしていないため、効果はありません。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@dataclass
class Config:
    model_name = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
    weights_path = '/kaggle/input/lmsys-model/model'
    max_length = 2048
    batch_size = 4
    device = torch.device("cuda")    
    tta = False  # test time augmentation. <prompt>-<model-b's response>-<model-a's response>
    spread_max_length = False  # whether to apply max_length//3 on each input or max_length on the concatenated input

cfg = Config()
```

</div>
<div class="column-right">

# 日本語訳

```python
@dataclass
class Config:
    model_name = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # モデルの名前
    weights_path = '/kaggle/input/lmsys-model/model'  # 重みのパス
    max_length = 2048  # 最大長
    batch_size = 4  # バッチサイズ
    device = torch.device("cuda")  # 使用するデバイス
    tta = False  # テストタイム拡張。<prompt>-<model-bの応答>-<model-aの応答>
    spread_max_length = False  # max_length//3を各入力に適用するか、連結入力にmax_lengthを適用するか

cfg = Config()
```

</div>
</details>

In [ ]:
@dataclass
class Config:
    model_name = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # モデルの名前
    weights_path = '/kaggle/input/lmsys-model/model'  # 重みのパス
    max_length = 2048  # 最大長
    batch_size = 4  # バッチサイズ
    device = torch.device("cuda")  # 使用するデバイス
    tta = False  # テストタイム拡張。<prompt>-<model-bの応答>-<model-aの応答>
    spread_max_length = False  # max_length//3を各入力に適用するか、連結入力にmax_lengthを適用するか

cfg = Config()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Prepare Data 

</div>
<div class="column-right">

# 日本語訳

# データの準備 


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# concatenate strings in list
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

display(test.head(5))
```

</div>
<div class="column-right">

# 日本語訳

```python
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# リスト内の文字列を連結する関数
def process(input_str):
    stripped_str = input_str.strip('[]')  # 文字列の先頭と末尾の角括弧を取り除く
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文字列を分割してトリミング
    return  ' '.join(sentences)  # 文を結合して返す

test.loc[:, 'prompt'] = test['prompt'].apply(process)  # 'prompt'列を処理
test.loc[:, 'response_a'] = test['response_a'].apply(process)  # 'response_a'列を処理
test.loc[:, 'response_b'] = test['response_b'].apply(process)  # 'response_b'列を処理

display(test.head(5))  # 上位5行を表示
```

</div>
</details>

In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')

# リスト内の文字列を連結する関数
def process(input_str):
    stripped_str = input_str.strip('[]')  # 文字列の先頭と末尾の角括弧を取り除く
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文字列を分割してトリミング
    return  ' '.join(sentences)  # 文を結合して返す

test.loc[:, 'prompt'] = test['prompt'].apply(process)  # 'prompt'列を処理
test.loc[:, 'response_a'] = test['response_a'].apply(process)  # 'response_a'列を処理
test.loc[:, 'response_b'] = test['response_b'].apply(process)  # 'response_b'列を処理

display(test.head(5))  # 上位5行を表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Tokenize

</div>
<div class="column-right">

# 日本語訳

# トークナイズ


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["User prompt: " + p for p in prompt]
    response_a = ["\n\nModel A :\n" + r_a for r_a in response_a]
    response_b = ["\n\n--------\n\nModel B:\n" + r_b for r_b in response_b]
    if spread_max_length:
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        attention_mask = [[1]* len(i) for i in input_ids]
    else:
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)
        input_ids = tokenized.input_ids
        attention_mask = tokenized.attention_mask
    return input_ids, attention_mask
```

</div>
<div class="column-right">

# 日本語訳

```python
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["User prompt: " + p for p in prompt]  # ユーザーのプロンプトを整形
    response_a = ["\n\nModel A :\n" + r_a for r_a in response_a]  # Model Aの応答を整形
    response_b = ["\n\n--------\n\nModel B:\n" + r_b for r_b in response_b]  # Model Bの応答を整形
    if spread_max_length:
        # max_lengthを各入力に均等に分配する場合
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # 入力IDの生成
        attention_mask = [[1]* len(i) for i in input_ids]  # 注意マスクの生成
    else:
        # max_lengthを全体の入力に適用する場合
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)  # トークナイザでトークナイズ
        input_ids = tokenized.input_ids  # 入力IDを取得
        attention_mask = tokenized.attention_mask  # 注意マスクを取得
    return input_ids, attention_mask  # 入力IDと注意マスクを返す
```

</div>
</details>

In [ ]:
def tokenize(
    tokenizer, prompt, response_a, response_b, max_length=cfg.max_length, spread_max_length=cfg.spread_max_length
):
    prompt = ["User prompt: " + p for p in prompt]  # ユーザーのプロンプトを整形
    response_a = ["\n\nModel A :\n" + r_a for r_a in response_a]  # Model Aの応答を整形
    response_b = ["\n\n--------\n\nModel B:\n" + r_b for r_b in response_b]  # Model Bの応答を整形
    if spread_max_length:
        # max_lengthを各入力に均等に分配する場合
        prompt = tokenizer(prompt, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_a = tokenizer(response_a, max_length=max_length//3, truncation=True, padding=False).input_ids
        response_b = tokenizer(response_b, max_length=max_length//3, truncation=True, padding=False).input_ids
        input_ids = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # 入力IDの生成
        attention_mask = [[1]* len(i) for i in input_ids]  # 注意マスクの生成
    else:
        # max_lengthを全体の入力に適用する場合
        text = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        tokenized = tokenizer(text, max_length=max_length, truncation=True, padding=False)  # トークナイザでトークナイズ
        input_ids = tokenized.input_ids  # 入力IDを取得
        attention_mask = tokenized.attention_mask  # 注意マスクを取得
    return input_ids, attention_mask  # 入力IDと注意マスクを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%time

tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

data = pd.DataFrame()
data["id"] = test["id"]
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])
data["length"] = data["input_ids"].apply(len)

aug_data = pd.DataFrame()
aug_data["id"] = test["id"]
# swap response_a & response_b
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)
```

</div>
<div class="column-right">

# 日本語訳

```python
%%time

tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')  # トークナイザーを読み込み

data = pd.DataFrame()
data["id"] = test["id"]  # ID列を作成
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])  # トークナイズを適用
data["length"] = data["input_ids"].apply(len)  # 入力IDの長さを計算

aug_data = pd.DataFrame()
aug_data["id"] = test["id"]  # ID列を作成
# response_aとresponse_bを入れ替える
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)  # 入力IDの長さを計算
```

</div>
</details>

In [ ]:
%%time

tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')  # トークナイザーを読み込み

data = pd.DataFrame()
data["id"] = test["id"]  # ID列を作成
data["input_ids"], data["attention_mask"] = tokenize(tokenizer, test["prompt"], test["response_a"], test["response_b"])  # トークナイズを適用
data["length"] = data["input_ids"].apply(len)  # 入力IDの長さを計算

aug_data = pd.DataFrame()
aug_data["id"] = test["id"]  # ID列を作成
# response_aとresponse_bを入れ替える
aug_data['input_ids'], aug_data['attention_mask'] = tokenize(tokenizer, test["prompt"], test["response_b"], test["response_a"])
aug_data["length"] = aug_data["input_ids"].apply(len)  # 入力IDの長さを計算

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print(tokenizer.decode(data["input_ids"][0]))
```

</div>
<div class="column-right">

# 日本語訳

```python
print(tokenizer.decode(data["input_ids"][0]))  # 最初の入力IDをデコードして表示
```

</div>
</details>

In [ ]:
print(tokenizer.decode(data["input_ids"][0]))  # 最初の入力IDをデコードして表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
print(tokenizer.decode(aug_data["input_ids"][0]))
```

</div>
<div class="column-right">

# 日本語訳

```python
print(tokenizer.decode(aug_data["input_ids"][0]))  # 最初の入れ替えた入力IDをデコードして表示
```

</div>
</details>

In [ ]:
print(tokenizer.decode(aug_data["input_ids"][0]))  # 最初の入れ替えた入力IDをデコードして表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Load model 
We load 1 model on each gpu.  

</div>
<div class="column-right">

# 日本語訳

# モデルの読み込み 
各GPUに1モデルを読み込みます。  


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# BitsAndBytes configuration
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=False,
)

# Load base model on GPU 0
device_0 = torch.device('cuda:0')
base_model_0 = LlamaForSequenceClassification.from_pretrained(
    cfg.model_name,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:0')
base_model_0.config.pad_token_id = tokenizer.pad_token_id

# Load base model on GPU 1
device_1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    cfg.model_name,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')
base_model_1.config.pad_token_id = tokenizer.pad_token_id
```

</div>
<div class="column-right">

# 日本語訳

```python
# BitsAndBytesの設定
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,  # モデルを8ビットで読み込む設定
    bnb_8bit_compute_dtype=torch.float16,  # 計算のデータ型をfloat16に設定
    bnb_8bit_use_double_quant=False,  # 二重量子化を無効にする設定
)

# GPU 0にベースモデルを読み込む
device_0 = torch.device('cuda:0')
base_model_0 = LlamaForSequenceClassification.from_pretrained(
    cfg.model_name,
    num_labels=3,
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:0')  # GPU 0に割り当て
base_model_0.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定

# GPU 1にベースモデルを読み込む
device_1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    cfg.model_name,
    num_labels=3,
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:1')  # GPU 1に割り当て
base_model_1.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定
```

</div>
</details>

In [ ]:
# BitsAndBytesの設定
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,  # モデルを8ビットで読み込む設定
    bnb_8bit_compute_dtype=torch.float16,  # 計算のデータ型をfloat16に設定
    bnb_8bit_use_double_quant=False,  # 二重量子化を無効にする設定
)

# GPU 0にベースモデルを読み込む
device_0 = torch.device('cuda:0')
base_model_0 = LlamaForSequenceClassification.from_pretrained(
    cfg.model_name,
    num_labels=3,
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:0')  # GPU 0に割り当て
base_model_0.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定

# GPU 1にベースモデルを読み込む
device_1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    cfg.model_name,
    num_labels=3,
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,  # 量子化設定を適用
    device_map='cuda:1')  # GPU 1に割り当て
base_model_1.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Load weights 

</div>
<div class="column-right">

# 日本語訳

# 重みの読み込み 


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.10,
    bias='none',
    inference_mode=True,
    task_type=TaskType.SEQ_CLS,
    target_modules=['o_proj', 'v_proj']
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# LoRAの設定
peft_config = LoraConfig(
    r=16,  # 縮小次元
    lora_alpha=32,  # LoRAのスケーリング係数
    lora_dropout=0.10,  # ドロップアウト率
    bias='none',  # バイアスの設定
    inference_mode=True,  # 推論モードを有効にする
    task_type=TaskType.SEQ_CLS,  # タスクの種類
    target_modules=['o_proj', 'v_proj']  # 対象モジュール
)
```

</div>
</details>

In [ ]:
# LoRAの設定
peft_config = LoraConfig(
    r=16,  # 縮小次元
    lora_alpha=32,  # LoRAのスケーリング係数
    lora_dropout=0.10,  # ドロップアウト率
    bias='none',  # バイアスの設定
    inference_mode=True,  # 推論モードを有効にする
    task_type=TaskType.SEQ_CLS,  # タスクの種類
    target_modules=['o_proj', 'v_proj']  # 対象モジュール
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Get peft
model_0 = get_peft_model(base_model_0, peft_config).to(device_0) 
# Load weights
model_0.load_state_dict(torch.load(cfg.weights_path), strict=False)
model_0.eval()

model_1 = get_peft_model(base_model_1, peft_config).to(device_1)
model_1.load_state_dict(torch.load(cfg.weights_path), strict=False)
model_1.eval()
```

</div>
<div class="column-right">

# 日本語訳

```python
# PEFTの取得
model_0 = get_peft_model(base_model_0, peft_config).to(device_0)  # PEFTモデルを取得してGPU 0に配置
# 重みを読み込む
model_0.load_state_dict(torch.load(cfg.weights_path), strict=False)
model_0.eval()  # 評価モードに設定

model_1 = get_peft_model(base_model_1, peft_config).to(device_1)  # PEFTモデルを取得してGPU 1に配置
model_1.load_state_dict(torch.load(cfg.weights_path), strict=False)  # 重みを読み込む
model_1.eval()  # 評価モードに設定
```

</div>
</details>

In [ ]:
# PEFTの取得
model_0 = get_peft_model(base_model_0, peft_config).to(device_0)  # PEFTモデルを取得してGPU 0に配置
# 重みを読み込む
model_0.load_state_dict(torch.load(cfg.weights_path), strict=False)
model_0.eval()  # 評価モードに設定

model_1 = get_peft_model(base_model_1, peft_config).to(device_1)  # PEFTモデルを取得してGPU 1に配置
model_1.load_state_dict(torch.load(cfg.weights_path), strict=False)  # 重みを読み込む
model_1.eval()  # 評価モードに設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Trainable Parameters
model_0.print_trainable_parameters()
model_1.print_trainable_parameters()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 学習可能なパラメータ
model_0.print_trainable_parameters()  # モデル0の学習可能なパラメータを表示
model_1.print_trainable_parameters()  # モデル1の学習可能なパラメータを表示
```

</div>
</details>

In [ ]:
# 学習可能なパラメータ
model_0.print_trainable_parameters()  # モデル0の学習可能なパラメータを表示
model_1.print_trainable_parameters()  # モデル1の学習可能なパラメータを表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Inference


</div>
<div class="column-right">

# 日本語訳

# 推論



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        tmp = df.iloc[start_idx:end_idx]
        input_ids = tmp["input_ids"].to_list()
        attention_mask = tmp["attention_mask"].to_list()
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",
            pad_to_multiple_of=None,
            return_tensors="pt",
        )
        outputs = model(**inputs.to(device))
        proba = outputs.logits.softmax(-1).cpu()
        
        a_win.extend(proba[:, 0].tolist())
        b_win.extend(proba[:, 1].tolist())
        tie.extend(proba[:, 2].tolist())
    
    df["winner_model_a"] = a_win
    df["winner_model_b"] = b_win
    df["winner_tie"] = tie
    
    return df
```

</div>
<div class="column-right">

# 日本語訳

```python
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝ちと引き分けのカウントを初期化
    
    # バッチサイズに基づいてデータを分割して推論を行う
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりのインデックスを決定
        tmp = df.iloc[start_idx:end_idx]  # データフレームのサブセットを取得
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換
        attention_mask = tmp["attention_mask"].to_list()  # 注意マスクをリストに変換
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",  # 最長の長さでパディング
            pad_to_multiple_of=None,
            return_tensors="pt",  # PyTorchテンソルとして返す
        )
        outputs = model(**inputs.to(device))  # モデルに入力を渡して出力を計算
        proba = outputs.logits.softmax(-1).cpu()  # ロジットをソフトマックス関数で処理
        
        a_win.extend(proba[:, 0].tolist())  # Model Aの勝ちの確率を追加
        b_win.extend(proba[:, 1].tolist())  # Model Bの勝ちの確率を追加
        tie.extend(proba[:, 2].tolist())  # 引き分けの確率を追加
    
    df["winner_model_a"] = a_win  # データフレームにModel Aの勝ちの確率を追加
    df["winner_model_b"] = b_win  # データフレームにModel Bの勝ちの確率を追加
    df["winner_tie"] = tie  # データフレームに引き分けの確率を追加
    
    return df  # 結果のデータフレームを返す
```

</div>
</details>

In [ ]:
@torch.no_grad()
@torch.cuda.amp.autocast()
def inference(df, model, device, batch_size=cfg.batch_size, max_length=cfg.max_length):
    a_win, b_win, tie = [], [], []  # 各モデルの勝ちと引き分けのカウントを初期化
    
    # バッチサイズに基づいてデータを分割して推論を行う
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりのインデックスを決定
        tmp = df.iloc[start_idx:end_idx]  # データフレームのサブセットを取得
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換
        attention_mask = tmp["attention_mask"].to_list()  # 注意マスクをリストに変換
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",  # 最長の長さでパディング
            pad_to_multiple_of=None,
            return_tensors="pt",  # PyTorchテンソルとして返す
        )
        outputs = model(**inputs.to(device))  # モデルに入力を渡して出力を計算
        proba = outputs.logits.softmax(-1).cpu()  # ロジットをソフトマックス関数で処理
        
        a_win.extend(proba[:, 0].tolist())  # Model Aの勝ちの確率を追加
        b_win.extend(proba[:, 1].tolist())  # Model Bの勝ちの確率を追加
        tie.extend(proba[:, 2].tolist())  # 引き分けの確率を追加
    
    df["winner_model_a"] = a_win  # データフレームにModel Aの勝ちの確率を追加
    df["winner_model_b"] = b_win  # データフレームにModel Bの勝ちの確率を追加
    df["winner_tie"] = tie  # データフレームに引き分けの確率を追加
    
    return df  # 結果のデータフレームを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

# sort by input length to fully leverage dynaminc padding
data = data.sort_values("length", ascending=False)
# the total #tokens in sub_1 and sub_2 should be more or less the same
sub_1 = data.iloc[0::2].copy()
sub_2 = data.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

result_df = pd.concat(list(results), axis=0)
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values

print(f"elapsed time: {time.time() - st}")
```

</div>
<div class="column-right">

# 日本語訳

```python
st = time.time()

# 动的パディングを最大限に活用するために入力長でソート
data = data.sort_values("length", ascending=False)
# sub_1とsub_2のトークン数がほぼ同じである必要があります
sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをコピー
sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをコピー

with ThreadPoolExecutor(max_workers=2) as executor:  # スレッドプールを使って並列処理
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 並列に推論を実行

result_df = pd.concat(list(results), axis=0)  # 結果を結合
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 勝者の確率を取得

print(f"経過時間: {time.time() - st}")  # 経過時間を表示
```

</div>
</details>

In [ ]:
st = time.time()

# 动的パディングを最大限に活用するために入力長でソート
data = data.sort_values("length", ascending=False)
# sub_1とsub_2のトークン数がほぼ同じである必要があります
sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをコピー
sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをコピー

with ThreadPoolExecutor(max_workers=2) as executor:  # スレッドプールを使って並列処理
    results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 並列に推論を実行

result_df = pd.concat(list(results), axis=0)  # 結果を結合
proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 勝者の確率を取得

print(f"経過時間: {time.time() - st}")  # 経過時間を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
st = time.time()

if cfg.tta:
    data = aug_data.sort_values("length", ascending=False)  # sort by input length to boost speed
    sub_1 = data.iloc[0::2].copy()
    sub_2 = data.iloc[1::2].copy()

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))

    tta_result_df = pd.concat(list(results), axis=0)
    # recall TTA's order is flipped
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values 
    # average original result and TTA result.
    proba = (proba + tta_proba) / 2

print(f"elapsed time: {time.time() - st}")
```

</div>
<div class="column-right">

# 日本語訳

```python
st = time.time()

if cfg.tta:
    data = aug_data.sort_values("length", ascending=False)  # 入力長でソートして速度を向上させる
    sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをコピー
    sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをコピー

    with ThreadPoolExecutor(max_workers=2) as executor:  # スレッドプールを使って並列処理
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 並列に推論を実行

    tta_result_df = pd.concat(list(results), axis=0)  # 結果を結合
    # TTAの結果は順序が逆であることに注意
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values  # TTAの確率を取得
    # 元の結果とTTAの結果を平均化する
    proba = (proba + tta_proba) / 2  # 確率を平均化

print(f"経過時間: {time.time() - st}")  # 経過時間を表示
```

</div>
</details>

In [ ]:
st = time.time()

if cfg.tta:
    data = aug_data.sort_values("length", ascending=False)  # 入力長でソートして速度を向上させる
    sub_1 = data.iloc[0::2].copy()  # 偶数インデックスのデータをコピー
    sub_2 = data.iloc[1::2].copy()  # 奇数インデックスのデータをコピー

    with ThreadPoolExecutor(max_workers=2) as executor:  # スレッドプールを使って並列処理
        results = executor.map(inference, (sub_1, sub_2), (model_0, model_1), (device_0, device_1))  # 並列に推論を実行

    tta_result_df = pd.concat(list(results), axis=0)  # 結果を結合
    # TTAの結果は順序が逆であることに注意
    tta_proba = tta_result_df[["winner_model_b", "winner_model_a", "winner_tie"]].values  # TTAの確率を取得
    # 元の結果とTTAの結果を平均化する
    proba = (proba + tta_proba) / 2  # 確率を平均化

print(f"経過時間: {time.time() - st}")  # 経過時間を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
result_df.loc[:, "winner_model_a"] = proba[:, 0]
result_df.loc[:, "winner_model_b"] = proba[:, 1]
result_df.loc[:, "winner_tie"] = proba[:, 2]
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]
submission_df.to_csv('submission.csv', index=False)
display(submission_df)
```

</div>
<div class="column-right">

# 日本語訳

```python
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # データフレームにModel Aの勝ちの確率を追加
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # データフレームにModel Bの勝ちの確率を追加
result_df.loc[:, "winner_tie"] = proba[:, 2]  # データフレームに引き分けの確率を追加
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  # 提出用データフレームを作成
submission_df.to_csv('submission.csv', index=False)  # CSVファイルに書き出し
display(submission_df)  # 提出データを表示
```

</div>
</details>

In [ ]:
result_df.loc[:, "winner_model_a"] = proba[:, 0]  # データフレームにModel Aの勝ちの確率を追加
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # データフレームにModel Bの勝ちの確率を追加
result_df.loc[:, "winner_tie"] = proba[:, 2]  # データフレームに引き分けの確率を追加
submission_df = result_df[["id", 'winner_model_a', 'winner_model_b', 'winner_tie']]  # 提出用データフレームを作成
submission_df.to_csv('submission.csv', index=False)  # CSVファイルに書き出し
display(submission_df)  # 提出データを表示